# Custom Object detector for the Gate Task

## Imports

In [0]:
import os
import numpy as np
import cv2
import pandas as pd

In [0]:
from keras import Input, layers
from keras.models import Sequential, Model
from keras import layers
from keras import Input
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator 

Using TensorFlow backend.


## Input the Images

In [0]:
image_dir = '/train_dir'
size_dim = (228,228)

In [0]:
def get_images(image_dir, size_dim=(228,228)):
    image_folder = image_dir
    count = 0
    data = []
    for file_name in os.list_dir(image_folder): 
        img = cv2.imread(file_name)
        img = cv2.resize(img, size_dim, interpolation = cv2.INTER_NEAREST)
        if img is None:
            print("Could not find the image. Some problem with image number: %d" %(count))
            print("-----------Skkipping Image %d -----------" %(count))
        else:
            data.append(img)
            count += 1
    
    print("Total Number of images read %d" %(count))
    return(np.asrray(data), count)

## Formatting the Input 

In [0]:
train_datagen = ImageDataGenerator(rotation_range=30, width_shift_range=0.2, height_shift_range=0.2,
             zoom_range=0.2, horizontal_flip=True, vertical_flip=False,
             rescale=1./255, data_format='channels_last', dtype='float32')

In [0]:
test_datagen = ImageDataGenerator(rescale=1./255, dtype='float32')

In [0]:
# train_datagen.flow_from_directory('/train_dir', target_size=(228, 228), color_mode='rgb', classes=None, batch_size=32, shuffle=True, class_mode='categorical', seed=31)

In [0]:
# test_datagen.flow_from_directory(directory='test_dir', target_size=(228,228), color_mode='rgb', classes=None)

In [0]:
train_datagen.flow(x, y, batch_size=32, shuffle=True)

## Object Detection Model

In [0]:
input_shape = Input(shape = (228, 228, 3))

x = layers.SeparableConv2D(128, kernel_size=3)(input_shape)
x = layers.advanced_activations.LeakyReLU(alpha=0.3)(x)
x = layers.MaxPooling2D(pool_size = (2, 2)) (x)
x = layers.BatchNormalization() (x)
x = layers.Dropout(0.2) (x)

x = layers.SeparableConv2D(64, kernel_size=3)(x)
x = layers.advanced_activations.LeakyReLU(alpha=0.3)(x)
x = layers.MaxPooling2D(pool_size = (2, 2)) (x)
x = layers.BatchNormalization() (x)
x = layers.Dropout(0.3) (x)

x = layers.SeparableConv2D(32, kernel_size=3)(x)
x = layers.advanced_activations.LeakyReLU(alpha=0.3)(x)
x = layers.MaxPooling2D(pool_size = (2, 2)) (x)
x = layers.BatchNormalization() (x)
x = layers.Dropout(0.4) (x)

x = layers.Flatten()(x)
x = layers.Dense(256) (x)
x = layers.advanced_activations.LeakyReLU(alpha=0.3)(x)
cordinates = layers.Dense(4, use_bias='False') (x)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
regressor = Model(inputs=input_shape, outputs=cordinates)

In [0]:
regressor.summary()